# Assignment 1

### Fetch Data from service API's and upload and retrive it back from your snowflake account


#### Import the necessary libraries

In [68]:
import requests
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

#### Fetch data function to call the API's

In [70]:
def fetch_data(url):
    auth = ("", "") #API access credentials
    response = requests.get(url, auth=auth)
    if response.status_code == 200:
        return response.json().get('d', {}).get('results', [])
    return []

#### Call The fetch_data() function

In [72]:
# API URLs

inventory_url = "https://verona.cob.csuchico.edu:8045/odata/240/Inventory?$format=json"

sales_url = "https://verona.cob.csuchico.edu:8045/odata/240/Sales?$format=json"

market_url = "https://verona.cob.csuchico.edu:8045/odata/240/Marketing_Expenses?$format=json"
# Fetching data
inventory_data = fetch_data(inventory_url)
sales_data = fetch_data(sales_url)
market_data = fetch_data(market_url)
print("Data Fetched Succesfully")

Data Fetched Succesfully


#### Convert JSON response to Pandas Data Frame

In [74]:
inventory_df = pd.DataFrame(inventory_data).drop(columns=["__metadata"], errors="ignore")
sales_df = pd.DataFrame(sales_data).drop(columns=["__metadata"], errors="ignore")
market_df = pd.DataFrame(market_data).drop(columns=["__metadata"], errors="ignore")

print("\nInventory Data Frame - \n")
print(inventory_df.head())

print(inventory_df.dtypes)

print("\nSales Data Frame - \n")
print(sales_df.head())

print("\nMarket Data Frame - \n")
print(market_df.head())


Inventory Data Frame - 

        ID  ROW_ID PLANT SIM_ROUND SIM_STEP SIM_DATE      SIM_CALENDAR_DATE  \
0  .1~1548    1548    CC        04       11    04/11  /Date(1710115200000)/   
1  .1~1372    1372    CC        04       03    04/03  /Date(1709424000000)/   
2  .1~2054    2054    CC        05       14    05/14  /Date(1712102400000)/   
3  .1~1900    1900    CC        05       07    05/07  /Date(1711497600000)/   
4  .1~1680    1680    CC        04       17    04/17  /Date(1710633600000)/   

   SIM_PERIOD  SIM_ELAPSED_STEPS STORAGE_LOCATION MATERIAL_NUMBER  \
0          15                 71               02          CC-F12   
1          13                 63               02          CC-F12   
2          19                 94               02          CC-F12   
3          18                 87               02          CC-F12   
4          16                 77               02          CC-F12   

   MATERIAL_DESCRIPTION     MATERIAL_TYPE MATERIAL_CODE MATERIAL_SIZE  \
0  1kg Blue

#### Process your data frames convert necessary fields into numeric values

In [76]:
# Function to convert numeric columns
def convert_numeric(df, numeric_columns):
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, set errors='coerce' to handle errors
    return df

# Inventory Table Numeric Columns
inventory_numeric_columns = [
    "ROW_ID", "SIM_ROUND", "SIM_STEP", "SIM_PERIOD", "SIM_ELAPSED_STEPS", "STORAGE_LOCATION", "INVENTORY_OPENING_BALANCE"
]
inventory_df = convert_numeric(inventory_df, inventory_numeric_columns)
print(inventory_df.dtypes)

# Sales Table Numeric Columns
sales_numeric_columns = [
    "ROW_ID", "SIM_ROUND", "SIM_STEP", "SIM_PERIOD", "SIM_ELAPSED_STEPS", "SALES_ORDER_NUMBER",
    "LINE_ITEM", "STORAGE_LOCATION", "QUANTITY", "QUANTITY_DELIVERED", "NET_PRICE",
    "NET_VALUE", "COST", "CONTRIBUTION_MARGIN", "CONTRIBUTION_MARGIN_PCT"
]
sales_df = convert_numeric(sales_df, sales_numeric_columns)
print("\n",sales_df.dtypes)

# Market Table Numeric Columns
market_numeric_columns = ["ROW_ID", "SIM_PERIOD", "SIM_ELAPSED_STEPS", "AMOUNT"]
market_df = convert_numeric(market_df, market_numeric_columns)
print("\n",market_df.dtypes)


ID                            object
ROW_ID                         int64
PLANT                         object
SIM_ROUND                      int64
SIM_STEP                       int64
SIM_DATE                      object
SIM_CALENDAR_DATE             object
SIM_PERIOD                     int64
SIM_ELAPSED_STEPS              int64
STORAGE_LOCATION               int64
MATERIAL_NUMBER               object
MATERIAL_DESCRIPTION          object
MATERIAL_TYPE                 object
MATERIAL_CODE                 object
MATERIAL_SIZE                 object
MATERIAL_LABEL                object
INVENTORY_OPENING_BALANCE    float64
UNIT                          object
dtype: object

 ID                          object
ROW_ID                       int64
SALES_ORGANIZATION          object
SIM_ROUND                    int64
SIM_STEP                     int64
SIM_DATE                    object
SIM_CALENDAR_DATE           object
SIM_PERIOD                   int64
SIM_ELAPSED_STEPS            int64
SAL

#### Connect to your snowflake account using snowflake connector library

In [78]:
conn = snowflake.connector.connect(
    user='ksimmonds', # Username
    password='Purposeful_2025#', # Password to your snowflake account
    account='xxqojql-kc79258', # Account URL ID
    warehouse='COMPUTE_WH', # Warhouse ID
    database='TEST', # Database Name
    schema='PUBLIC', # Schema Type Def - PUBLIC
    role='ACCOUNTADMIN' # Access role - ACCOUNTADMIN
    )

#### Create a cursor to your database and  create 3 new tables using SQL query

In [80]:
cursor=conn.cursor()

cursor.execute('''
    CREATE OR REPLACE TABLE "InventoryTable" (
        ID VARCHAR(16777216),
        ROW_ID NUMBER(38,0),
        PLANT VARCHAR(16777216),
        SIM_ROUND NUMBER(38,0),
        SIM_STEP NUMBER(38,0),
        SIM_DATE VARCHAR(16777216),
        SIM_CALENDAR_DATE VARCHAR(16777216),
        SIM_PERIOD NUMBER(38,0),
        SIM_ELAPSED_STEPS NUMBER(38,0),
        STORAGE_LOCATION NUMBER(38,0),
        MATERIAL_NUMBER VARCHAR(16777216),
        MATERIAL_DESCRIPTION VARCHAR(16777216),
        MATERIAL_TYPE VARCHAR(16777216),
        MATERIAL_CODE VARCHAR(16777216),
        MATERIAL_SIZE VARCHAR(16777216),
        MATERIAL_LABEL VARCHAR(16777216),
        INVENTORY_OPENING_BALANCE NUMBER(38,3),
        UNIT VARCHAR(16777216)
    );
''')

cursor.execute('''
    CREATE OR REPLACE TABLE "SalesTable" (
        ID VARCHAR(16777216),
        ROW_ID NUMBER(38,0),
        SALES_ORGANIZATION VARCHAR(16777216),
        SIM_ROUND VARCHAR(16777216),
        SIM_STEP VARCHAR(16777216),
        SIM_DATE VARCHAR(16777216),
        SIM_CALENDAR_DATE VARCHAR(16777216),
        SIM_PERIOD NUMBER(38,0),
        SIM_ELAPSED_STEPS NUMBER(38,0),
        SALES_ORDER_NUMBER VARCHAR(16777216),
        LINE_ITEM VARCHAR(16777216),
        STORAGE_LOCATION VARCHAR(16777216),
        REGION VARCHAR(16777216),
        AREA VARCHAR(16777216),
        CITY VARCHAR(16777216),
        COUNTRY VARCHAR(16777216),
        POSTAL_CODE VARCHAR(16777216),
        CUSTOMER_NUMBER VARCHAR(16777216),
        DISTRIBUTION_CHANNEL VARCHAR(16777216),
        MATERIAL_NUMBER VARCHAR(16777216),
        MATERIAL_DESCRIPTION VARCHAR(16777216),
        MATERIAL_TYPE VARCHAR(16777216),
        MATERIAL_CODE VARCHAR(16777216),
        MATERIAL_SIZE VARCHAR(16777216),
        MATERIAL_LABEL VARCHAR(16777216),
        QUANTITY NUMBER(38,3),
        QUANTITY_DELIVERED NUMBER(38,3),
        UNIT VARCHAR(16777216),
        NET_PRICE NUMBER(38,2),
        NET_VALUE NUMBER(38,2),
        COST NUMBER(38,2),
        CURRENCY VARCHAR(16777216),
        CONTRIBUTION_MARGIN NUMBER(38,5),
        CONTRIBUTION_MARGIN_PCT NUMBER(38,5)
    );
''')

cursor.execute('''
    CREATE OR REPLACE TABLE "MarketTable" (
        ID VARCHAR(16777216),
        ROW_ID NUMBER(38,0),
        SALES_ORGANIZATION VARCHAR(16777216),
        SIM_ROUND VARCHAR(16777216),
        SIM_STEP VARCHAR(16777216),
        SIM_DATE VARCHAR(50),
        SIM_CALENDAR_DATE VARCHAR(16777216),
        SIM_PERIOD NUMBER(38,0),
        SIM_ELAPSED_STEPS NUMBER(38,0),
        MATERIAL_NUMBER VARCHAR(16777216),
        MATERIAL_DESCRIPTION VARCHAR(16777216),
        AREA VARCHAR(16777216),
        AMOUNT NUMBER(38,2),
        CURRENCY VARCHAR(10)
    );
''')


#### Using the Snowflake pandas tool upload your data to their respective tables

In [82]:
write_pandas(conn,inventory_df,table_name="InventoryTable")
print("Inventory data uploaded")

write_pandas(conn,market_df,table_name="MarketTable")
print("Market data uploaded")

write_pandas(conn,sales_df,table_name="SalesTable")
print("Sales data uploaded")

Inventory data uploaded
Market data uploaded
Sales data uploaded


#### Try Fetching the data uploaded to the tables from snowflake using SQL query

In [84]:
sqlquery='SELECT * FROM "InventoryTable"'
cursor.execute(sqlquery)
resp=cursor.fetchall()
sf_data=pd.DataFrame(resp)
print(sf_data.head())

        0     1   2   3   4      5                      6   7   8   9   \
0  .1~1548  1548  CC   4  11  04/11  /Date(1710115200000)/  15  71   2   
1  .1~1372  1372  CC   4   3  04/03  /Date(1709424000000)/  13  63   2   
2  .1~2054  2054  CC   5  14  05/14  /Date(1712102400000)/  19  94   2   
3  .1~1900  1900  CC   5   7  05/07  /Date(1711497600000)/  18  87   2   
4  .1~1680  1680  CC   4  17  04/17  /Date(1710633600000)/  16  77   2   

       10                    11                12   13   14                15  \
0  CC-F12  1kg Blueberry Muesli  Finished Product  F12  1kg  Blueberry Muesli   
1  CC-F12  1kg Blueberry Muesli  Finished Product  F12  1kg  Blueberry Muesli   
2  CC-F12  1kg Blueberry Muesli  Finished Product  F12  1kg  Blueberry Muesli   
3  CC-F12  1kg Blueberry Muesli  Finished Product  F12  1kg  Blueberry Muesli   
4  CC-F12  1kg Blueberry Muesli  Finished Product  F12  1kg  Blueberry Muesli   

      16  17  
0  0.000  ST  
1  0.000  ST  
2  0.000  ST  
3  0.000